# Azure AI Search Knowledge Base with Knowledge Sources

In Azure AI Search, a **knowledge base** is a top-level resource that connects a chat completion model to one or more **knowledge sources** (searchable indexes) for use in agentic retrieval workloads. It defines which data sources to query, which model to use for reasoning, and how query execution should be optimized, powering the `retrieve` method in an LLM-driven information retrieval pipeline. Once created, a knowledge base can be updated at any time, and changes will take effect the next time it’s used. 

This notebook demonstrates how to build **Knowledge Bases** with **knowledge sources**. You’ll learn how agentic retrieval works, how to create and configure a knowledge base, and how to query it for grounded, citation-backed answers while exploring the reasoning behind each step.

## What You'll Learn

In this lab, you will:
* Connect to the existing knowledge sources that contain indexed documents
* Create a knowledge base that uses Azure OpenAI models for intelligent retrieval
* Query the knowledge base with natural language to get grounded, citation-backed answers
* Explore the agentic knowledge base's activity and reasoning process

## Architecture Overview

The lab environment has already provisioned:
* **Azure AI Search** - Contains two pre-indexed collections: 
  - **hrdocs** (50 documents): HR policies, employee handbook, company information
  - **healthdocs** (334 documents): Health benefits, insurance plans
* **Azure OpenAI** - Provides chat completion (gpt-5-mini) and embedding models (text-embedding-3-large)
* **Pre-computed vectors** - All documents are already vectorized and indexed

The indexes use **pre-computed embeddings**, meaning vector representations were generated during the setup process. This allows you to focus on the agentic retrieval layer without worrying about document ingestion or vectorization.

Let's get started! 🚀

## Step 1: Load Environment Variables

> **‼️ Important:** Make sure to open the project on VSCode from  `ignite25-LAB511-build-agentic-knowledge-bases...` folder. Opening it from `LAB511` will cause dependency errors.

Before we begin, we need to load the configuration for your Azure resources. The lab setup has created a `.env` file at the workspace root containing:

* Azure AI Search endpoint and credentials  
* Azure OpenAI endpoint, API key, and model deployment names  
* Knowledge base name

Run the cell below to load these environment variables into your Python session.

> **📌 Tip** 
> - The first time you run the cell below, you'll be prompted to select Kernel, select **Install/Enable suggested extensions**, then select **Python Environments** and finally choose the **.venv(3.11.9)** environment that is created for you.
> - You will also be prompted with "Do you want to allow public and private networks to access this app?" Select **Allow**.

In [1]:
# Not needed once we move to SDK
%load_ext autoreload
%autoreload 2

In [2]:
from dotenv import load_dotenv
from azure.identity.aio import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
import os

load_dotenv(override=True) # take environment variables from .env.

# Azure AI Search configuration
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_ADMIN_KEY")) if os.getenv("AZURE_SEARCH_ADMIN_KEY") else DefaultAzureCredential()
api_key = os.getenv("AZURE_SEARCH_ADMIN_KEY")

# Knowledge agent name
knowledge_agent_name = os.getenv("AZURE_SEARCH_KNOWLEDGE_AGENT", "knowledge-base")

# Azure OpenAI configuration
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model_name = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_NAME", "text-embedding-3-large")
azure_openai_chatgpt_deployment = os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT", "gpt-5-mini")
azure_openai_chatgpt_model_name = os.getenv("AZURE_OPENAI_CHATGPT_MODEL_NAME", "gpt-5-mini")

# Add a message to indicate that the environment variables have been loaded
print("Environment variables loaded. You can now create the knowledge sources and knowledge base.")

Environment variables loaded. You can now create the knowledge sources and knowledge base.


## Step 2: Create Knowledge Sources from Existing Indexes

A **Knowledge Source** in Azure AI Search is a pointer to a searchable index that the knowledge agent can query. In this lab, we're using the **SearchIndexKnowledgeSource** type, which wraps existing search indexes.

You have two pre-created indexes:
* **hrdocs** (50 documents) - HR policies, employee handbook, role library, company overview
* **healthdocs** (334 documents) - Health insurance plans, benefits options, coverage details

Run the cell below to create two knowledge sources that reference these indexes. This operation is fast because the indexes already exist—we're just creating metadata wrappers that the agent will use.

In [3]:
from lib import create_or_update_knowledge_source, create_or_update_knowledge_base

# Using REST helper functions instead of SDK objects to create knowledge sources and a knowledge base.
# Define knowledge source bodies (searchIndex kind) for existing indexes
hrdocs_body = {
    "name": "hrdocs-knowledge-source",
    "kind": "searchIndex",
    "description": "HR documents search index knowledge source",
    "searchIndexParameters": {
        "searchIndexName": "hrdocs"
    }
}

healthdocs_body = {
    "name": "healthdocs-knowledge-source",
    "kind": "searchIndex",
    "description": "Health documents search index knowledge source",
    "searchIndexParameters": {
        "searchIndexName": "healthdocs"
    }
}



print("Creating or updating knowledge sources via REST API helpers...")
hr_result = await create_or_update_knowledge_source(endpoint, api_key, hrdocs_body["name"], body=hrdocs_body)
print(f"✅ Upserted knowledge source: {hrdocs_body['name']}")

health_result = await create_or_update_knowledge_source(endpoint, api_key, healthdocs_body["name"], body=healthdocs_body)
print(f"✅ Upserted knowledge source: {healthdocs_body['name']}")

Creating or updating knowledge sources via REST API helpers...
✅ Upserted knowledge source: hrdocs-knowledge-source
✅ Upserted knowledge source: healthdocs-knowledge-source


## Step 3: Create a Knowledge Base

A **Knowledge Base** is an intelligent layer that connects your indexed data to a language model. Instead of just returning search results, it plans queries, retrieves relevant data, and generates grounded answers.

In this step, we'll create a knowledge base that acts as an intelligent wrapper around your knowledge sources (hrdocs and healthdocs) and LLM deployment.

### Output Modality Options

The knowledge base supports two output modalities:
* **`EXTRACTIVE_DATA`** (default) - Returns exact content from your knowledge sources without generative alteration
* **`ANSWER_SYNTHESIS`** (used here) - Generates natural language answers using the LLM that cite the retrieved content

We're using `ANSWER_SYNTHESIS` with `include_activity=True` to get:
* **LLM-generated answers** that cite retrieved documents
* **Detailed activity logs** showing the knowledge base's reasoning process, including subqueries and re-ranking decisions

Learn more about [answer synthesis in Azure AI Search](https://learn.microsoft.com/azure/search/search-agentic-retrieval-how-to-synthesize).

Run the cell below to create the knowledge base.

In [4]:
# Knowledge base (preview) body using azureOpenAI model config
knowledge_base_body = {
    "name": knowledge_agent_name,  # Reuse existing variable; acts as knowledge base name
    "description": "Lab knowledge base combining hrdocs and healthdocs indexes",
    "knowledgeSources": [
        {"name": hrdocs_body["name"]},
        {"name": healthdocs_body["name"]}
    ],
    "models": [
        {
            "kind": "azureOpenAI",
            "azureOpenAIParameters": {
                "deploymentId": azure_openai_chatgpt_deployment,
                "modelName": azure_openai_chatgpt_model_name,
                "resourceUri": azure_openai_endpoint,
                "apiKey": azure_openai_key
            }
        }
    ],
    "retrievalReasoningEffort": {"kind": "low"},
    "outputMode": "answerSynthesis"
}

print("Creating or updating knowledge base (preview resource)...")
kb_result = await create_or_update_knowledge_base(endpoint, api_key, knowledge_base_body["name"], body=knowledge_base_body)
print(f"✅ Upserted knowledge base: {knowledge_base_body['name']}")

Creating or updating knowledge base (preview resource)...
✅ Upserted knowledge base: knowledge-base


## Step 4: Query the Knowledge Base

Now it’s time to query our documents and see agentic retrieval in action. This step demonstrates how the knowledge base processes queries to produce grounded, citation-backed answers.

### How Agentic Retrieval Works

Given a user query and conversation history, the knowledge base:
1. **Analyzes the conversation** - Understands the full context and user's information need
2. **Query decomposition** - Breaks down complex queries into focused subqueries
3. **Concurrent execution** - Runs subqueries in parallel against your knowledge source
4. **Semantic reranking** - Uses semantic ranker to rerank and filter results for relevance
5. **Answer synthesis** - Synthesizes the top results into a natural-language answer with citations

### Example Query

The example below asks about differences between two employee benefit plans. The knowledge agent will:
* Search across both indexed collections (384 documents total)
* Find relevant sections in the multi-index setup
* Generate a comparative answer with specific citations


Try running the cell below, then modify the `text` field in the `messages` list to ask your own questions!

In [ ]:
from lib import retrieve_from_knowledge_base
import warnings, os, time
warnings.filterwarnings('ignore', category=ResourceWarning)

# Build base retrieve body.
retrieve_body = {
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Differences between Northwind Health Plus and Standard"}
            ]
        }
    ],
    "includeActivity": True,
    "knowledgeSourceParams": [
        {
            "knowledgeSourceName": "hrdocs-knowledge-source",
            "kind": "searchIndex",
            "includeReferences": True,
            "includeReferenceSourceData": True,
            "alwaysQuerySource": True
        },
        {
            "knowledgeSourceName": "healthdocs-knowledge-source",
            "kind": "searchIndex",
            "includeReferences": True,
            "includeReferenceSourceData": True,
            "alwaysQuerySource": True
        }
    ]
}


raw_result = await retrieve_from_knowledge_base(
    endpoint,
    api_key,
    knowledge_agent_name,
    body=retrieve_body
)
print("\n✅ Retrieved response from knowledge base.\n")


📝 Synthesized answer preview:

Brief summary of differences between Northwind Health Plus and Northwind Standard:

- Coverage scope: Northwind Health Plus is a comprehensive plan covering medical, vision, dental, prescription drugs, mental health and substance-abuse services, and preventive care [ref_id:0][ref_id:3].
- Emergency and out-of-network services: Plus covers emergency services both in-network and out-of-network [ref_id:0][ref_id:3].
- Standard’s scope: Northwind Standard is a more basic plan covering medical, vision, dental, preventive care, and prescription drugs [ref_id:4][ref_id:9].
- Standard limits: Several documents state Standard does not offer emergency services, mental health/substance-abuse coverage, or out-of-network services [ref_id:4][ref_id:9][ref_id:5].
- Conflicting note about mental health: one source indicates Northwind Standard covers professional visits including mental health services, diagnostic tests, and related services [ref_id:1].
- Prescription dr

## Step 5: Review the Retrieval Response

The knowledge base's response contains three key components that provide full transparency into the retrieval process:

### 1. Response Content (Answer)
An LLM-generated answer to your query that cites the retrieved documents. This is the primary output you'd show to end users.

### 2. Activity Content (Reasoning)
Detailed planning and execution information showing:
* **Subqueries generated** - How the agentic knowledge base broke down your query
* **Reranking decisions** - Which results were promoted or filtered
* **Intermediate steps** - The agentic knowledge base's thought process and execution flow

### 3. References Content (Sources)
Source documents and text chunks that contributed to the answer, including:
* Document names and metadata
* Specific text passages used
* Relevance scores and rankings

### Why This Matters

These three components enable you to:
* **Verify grounding** - Ensure answers are based on your actual content
* **Build traceable citations** - Create links back to source documents
* **Debug and optimize** - Understand why certain results were retrieved
* **Tune retrieval parameters** - Adjust reranker thresholds and knowledge source settings

> **💡 Tip:** Retrieval parameters (like reranker thresholds and knowledge source parameters) influence how aggressively your agentic knowledge base reranks results and which sources it queries. Inspect the activity and references to validate grounding quality.

Let's examine each component! 

Run the first cell below to display the **synthesized answer** with citations:

In [10]:
response_items = raw_result.get("response", [])
if response_items and response_items[0].get("content"):
    first_content = response_items[0]["content"]
    if first_content and isinstance(first_content, list) and first_content[0].get("text"):
        print("\n📝 Synthesized answer:\n")
        print(first_content[0]["text"])
else:
    print("(No response content returned; inspect raw_result)")


📝 Synthesized answer:

Brief summary of differences between Northwind Health Plus and Northwind Standard:

- Coverage scope: Northwind Health Plus is a comprehensive plan covering medical, vision, dental, prescription drugs, mental health and substance-abuse services, and preventive care [ref_id:0][ref_id:3].
- Emergency and out-of-network services: Plus covers emergency services both in-network and out-of-network [ref_id:0][ref_id:3].
- Standard’s scope: Northwind Standard is a more basic plan covering medical, vision, dental, preventive care, and prescription drugs [ref_id:4][ref_id:9].
- Standard limits: Several documents state Standard does not offer emergency services, mental health/substance-abuse coverage, or out-of-network services [ref_id:4][ref_id:9][ref_id:5].
- Conflicting note about mental health: one source indicates Northwind Standard covers professional visits including mental health services, diagnostic tests, and related services [ref_id:1].
- Prescription drugs: Plu

Run the cell below to display the **agentic knowledge base's activity log** showing query decomposition and reasoning:

In [11]:
import json

# Activity -> JSON string direct from raw_result (pure JSON parsing)
activity_list = raw_result.get("activity", [])
activity_content = json.dumps(activity_list, indent=2)
print("activity_content:\n", activity_content, "\n")

activity_content:
 [
  {
    "type": "modelQueryPlanning",
    "id": 0,
    "inputTokens": 1310,
    "outputTokens": 598,
    "elapsedMs": 10388
  },
  {
    "type": "searchIndex",
    "id": 1,
    "knowledgeSourceName": "hrdocs-knowledge-source",
    "queryTime": "2025-10-18T21:25:16.296Z",
    "count": 0,
    "elapsedMs": 594,
    "searchIndexArguments": {
      "search": "Northwind Health Plus plan features and coverage",
      "filter": null,
      "sourceDataFields": [
        {
          "name": "snippet"
        },
        {
          "name": "uid"
        }
      ],
      "searchFields": [],
      "semanticConfigurationName": "semantic-configuration"
    }
  },
  {
    "type": "searchIndex",
    "id": 2,
    "knowledgeSourceName": "hrdocs-knowledge-source",
    "queryTime": "2025-10-18T21:25:16.573Z",
    "count": 0,
    "elapsedMs": 277,
    "searchIndexArguments": {
      "search": "Northwind Health Standard plan features and coverage",
      "filter": null,
      "sourceData

Run the cell below to display the **source references** used to generate the answer:

In [12]:
import json

# References -> JSON string direct from raw_result (pure JSON parsing)
references_list = raw_result.get("references", [])
references_content = json.dumps(references_list, indent=2)
print("references_content:\n", references_content, "\n")

references_content:
 [
  {
    "type": "searchIndex",
    "id": "0",
    "activitySource": 4,
    "sourceData": {
      "uid": "458e5142f311_aHR0cHM6Ly9tYWdvdHRlaWFkbHNnZW4yLmJsb2IuY29yZS53aW5kb3dzLm5ldC9oZWFsdGhkb2NzL05vcnRod2luZF9IZWFsdGhfUGx1c19CZW5lZml0c19EZXRhaWxzLnBkZg2_pages_23",
      "snippet": "PLAN INFORMATION \n\n  \n\n\n\nNorthwind Health Plus is a comprehensive health plan that offers coverage for medical, \n\nvision, and dental services. It also provides coverage for prescription drugs, mental health \n\nand substance abuse services, and preventive care. You can choose from a variety of in- \n\nnetwork providers, including primary care physicians, specialists, hospitals, and \n\npharmacies. Emergency services are also covered, both in-network and out-of-network. \n\nCo-pays, deductibles, and out-of-pocket maximums may apply to your plan. Your plan may \n\nalso include separate deductibles for different services, such as prescription drugs and \n\nhospitalization. It is i

## Notebook Complete! 🎉

Congratulations! You've successfully completed the hands-on portion of this lab.

### What You Learned: The Agentic Knowledge Base Orchestration Layer

This lab focused on the **intelligence layer** that sits on top of search infrastructure. Here's the real depth of what you explored:

#### **1. Knowledge Bases as Orchestrators**
You built a knowledge base that doesn't just search—it **orchestrates** an entire retrieval workflow. The knowledge base coordinates between your knowledge sources (the indexed blob data) and the LLM (GPT model) to produce grounded, citation-backed answers.

#### **2. Answer Synthesis vs. Raw Retrieval**
Traditional search returns chunks of text. You configured **Answer Synthesis mode**, where the LLM reads retrieved passages and generates a coherent natural-language response with inline citations. This is the foundation of modern copilots and chat interfaces over enterprise data.

#### **3. Query Decomposition & Planning**
When you examined the **activity logs**, you saw how the knowledge base breaks complex queries into focused subqueries, executes them in parallel, and applies semantic reranking. This is agentic behavior—the system plans and adapts based on the query context.

#### **4. Grounding & Transparency**
Every answer is backed by source references. You explored how to trace responses back to specific documents and passages, enabling trust and auditability in AI systems. The activity logs show exactly how the knowledge base arrived at its answer.

#### **5. The RAG Evolution**
This lab demonstrated **Retrieval-Augmented Generation (RAG)** at scale. The knowledge base doesn't just retrieve—it reasons about what to retrieve, how to combine results, and how to synthesize coherent answers. This is next-level RAG: agentic, explainable, and production-ready.

### Next Steps

📖 **Return to the lab instructions page** to have a look at the summary and reflect on how **agentic behavior elevates RAG beyond simple search**, explore additional learning resources, and understand how to apply these patterns to real-world enterprise AI solutions.

💡 **Experiment more!** Go back to Step 4 and try different queries—watch how the agentic knowledge base adapts its retrieval strategy based on query complexity.